In [1]:
# Import dependencies
import os
import pandas as pd
import numpy as np

In [2]:
# View Data folder files
os.listdir('Data')

['purchase_data.csv']

In [3]:
# Read CSV into Pandas data frame
purchase_data = pd.read_csv('Data/purchase_data.csv')

# Preview data frame
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [4]:
# Check for null values
display(purchase_data.count())

# Check for "-" values
print(purchase_data[purchase_data.isin(["-"])].count().sum())

# Check for infinity and negative infinity values
print(purchase_data[purchase_data.isin([np.inf])].count().sum())
print(purchase_data[purchase_data.isin([-np.inf])].count().sum())

Purchase ID    780
SN             780
Age            780
Gender         780
Item ID        780
Item Name      780
Price          780
dtype: int64

0
0
0


In [5]:
# Confirm proper data types
display(purchase_data.dtypes)

Purchase ID      int64
SN              object
Age              int64
Gender          object
Item ID          int64
Item Name       object
Price          float64
dtype: object

In [6]:
# Confirm 'Age' column values make sense
print(purchase_data['Age'].min())
print(purchase_data['Age'].max())

7
45


## Player Count

In [7]:
# Total number of players
total_players = purchase_data['SN'].nunique()

pd.DataFrame({'Total Players':[total_players]})

,Total Players
0,576


## Purchasing Analysis (Total)

In [8]:
# Number of unique items
unique_items = purchase_data['Item ID'].nunique()

# Average price
avg_price = purchase_data['Price'].mean()

# Total number of purchases
total_purchases = purchase_data['Purchase ID'].count()

# Total revenue
total_revenue = purchase_data['Price'].sum()

In [9]:
# Create summary data frame and format currency
purchasing_analysis = pd.DataFrame({'Number of Unique Items':[unique_items],
                                   'Average Price':"${:,.2f}".format(avg_price),
                                    'Number of Purchases':total_purchases,
                                    'Total Revenue':"${:,.2f}".format(total_revenue)
                                   })

# Display Purchasing Analysis (Total) summary data frame
purchasing_analysis

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,"$2,379.77"


## Gender Demographics

In [10]:
# Isolate columns of interest, drop duplicate 'SN' values and rename 'SN' column to 'Total Count'
unique_SN_df = purchase_data[['SN', 'Gender']].drop_duplicates(subset='SN').rename(columns={'SN':'Total Count'})

# Calculate number of players by gender
gender_demos = unique_SN_df.groupby('Gender').count()

# Create and calculate 'Percentage of Players' column and round values to two decimal points
gender_demos['Percentage of Players'] = round((gender_demos['Total Count'] / gender_demos['Total Count'].sum()) * 100, 2)

# Display Gender Demographics summary data frame
gender_demos

,Total Count,Percentage of Players
Gender,,
Female,81,14.06
Male,484,84.03
Other / Non-Disclosed,11,1.91


## Purchasing Analysis (Gender)

In [11]:
# Calculate purchase count by gender
gender_purchases = purchase_data[['Gender', 'Price']].groupby('Gender').count().rename(columns={'Price':'Purchase Count'})

# Create and calculate 'Total Purchase Value' column
gender_purchases['Total Purchase Value'] = purchase_data[['Gender', 'Price']].groupby('Gender').sum()

# Create and calculate 'Average Purchase Price' column
gender_purchases['Average Purchase Price'] = gender_purchases['Total Purchase Value'] / gender_purchases['Purchase Count']

# Create and calculate 'Avg Total Purchase per Person' column
gender_purchases['Avg Total Purchase per Person'] = gender_purchases['Total Purchase Value'] / gender_demos['Total Count']

# Display Purchasing Analysis (Gender) summary data frame
gender_purchases

,Purchase Count,Total Purchase Value,Average Purchase Price,Avg Total Purchase per Person
Gender,,,,
Female,113,361.94,3.203009,4.468395
Male,652,1967.64,3.017853,4.065372
Other / Non-Disclosed,15,50.19,3.346000,4.562727


In [12]:
# Reorder columns
gender_purchases = gender_purchases[['Purchase Count', 'Average Purchase Price', 'Total Purchase Value', 'Avg Total Purchase per Person']]

# Format currency columns
gender_purchases['Average Purchase Price'] = gender_purchases['Average Purchase Price'].map("${:,.2f}".format)
gender_purchases['Total Purchase Value'] = gender_purchases['Total Purchase Value'].map("${:,.2f}".format)
gender_purchases['Avg Total Purchase per Person'] = gender_purchases['Avg Total Purchase per Person'].map("${:,.2f}".format)

# Display Purchasing Analysis (Gender) summary data frame
gender_purchases

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

In [13]:
# Create age bins and labels
bins = [0, 9, 14, 19, 24, 29, 34, 39, 45]
age_ranges = ['<10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40+']

In [14]:
# Create data frame with age bins
age_data = purchase_data[['SN', 'Age']].drop_duplicates(subset='SN')
age_data['Age Bins'] = pd.cut(age_data['Age'], bins, labels=age_ranges)
age_demos = age_data.groupby('Age Bins').count()

# Display age bins data frame
age_demos

,SN,Age
Age Bins,,
<10,17,17
10-14,22,22
15-19,107,107
20-24,258,258
25-29,77,77
30-34,52,52
35-39,31,31
40+,12,12


In [15]:
# Remove unnecessary column
del age_demos['SN']

# Rename column 
age_demos.rename(columns={'Age':'Total Count'}, inplace=True)

# Create and calculate 'Percentage of Players' column
age_demos['Percentage of Players'] = round((age_demos['Total Count'] / age_demos['Total Count'].sum()) * 100, 2)

# Display Age Demographics summary table
age_demos

,Total Count,Percentage of Players
Age Bins,,
<10,17,2.95
10-14,22,3.82
15-19,107,18.58
20-24,258,44.79
25-29,77,13.37
30-34,52,9.03
35-39,31,5.38
40+,12,2.08


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Calculate the purchase count, average purchase price, total purchase value, and average total purchase per person


* Create a summary data frame to hold the results


* Display the summary data frame

## Top Spenders

* Calculate the purchase count, average purchase price, and total purchase value by SN


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Display a preview of the summary data frame

## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Calculate the purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Display a preview of the summary data frame

## Most Profitable Items

* Sort the Most Popular Items table by total purchase value in descending order


* Display a preview of the data frame